# Sentinel-3 data processing

## 0. Install and import packages

In [ ]:
!pip install cartopy
!pip install netCDF4
!pip install matplotlib-scalebar

In [ ]:
# a Google Eath engine python API is needed.
# See https://developers.google.com//earth-engine/tutorials/community/intro-to-python-api
project_name = str(input('Earth engine project name: '))
import ee ; ee.Authenticate() ; ee.Initialize(project=project_name)

In [ ]:
%load_ext autoreload
%autoreload 2
import os, time
import numpy as np
import numpy.ma as ma
import pandas as pd
from datetime import datetime as dt

import sys
sys.path.append("./Scripts/")
from Sentinel3Class import sentinel3
from Sentinel3_Utilities import (
    read_full_slstr_data,
    get_data_on_grid_of_interest,
    get_specific_date_files,
    perform_interpolation,
    set_nan_inf_neg_to_zero,
    get_inland_water_mask
    )

## 1. Inputs

Creates an input dictionary that includes all inputs necesary for processing each plume.

*   **plume_name**: name of the location, useful for filenaming for example
*   **cen_lon, cen_lat**: central longitude and latitude of the scene that is to be considered.
*   **delta_lonlat**: half of the width of the scene that is to be considered for interpolation
*   **pixel_size**: size of the new pixels, default is 0.001, which was found to work well.
*   **all_data_dir**: directory containg the interpolated data, both main and reference observations
*   **specific_main_dates**: a list with datetime objects defining with dates are to be considered as main dates, that is potentially containing methane plumes. There are three ways to define the specific_main_dates

The main dates can be specified in multiple ways through specific_main_dates:
1.  A list containing datetime date objects (excluding time stamps), [dt(y, m, d).date()]

        specific_main_dates = [dt(2020, 10, 11).date(), dt(2020, 10, 15).date()]

2. From a CSV file, with 'Days', 'Months', 'Years' as keys 

        csv_file = '/data/maartenvn/Varon_Algeria/S2_detected_plume_dates.csv'
        dates_df = pd.read_csv(csv_file)
        days = dates_df['Days']
        months = dates_df['Months']
        years = dates_df['Years']
        specific_main_dates = []
        for i in range(len(days)):
            specific_main_dates.append(dt(years[i], months[i], days[i]))

3. 'All': all dates/observations within the all_data_dir directory

        specific_main_dates = 'All'


In [ ]:
cen_lon = 6.17
cen_lat = 31.86

delta_lonlat = 0.2
pixel_size = 0.001

case_name = 'Case name' # Name of the data folder

original_file_dir = '/Data/'+ case_name + '/'

####################################
source = '/Data/'
####################################

####################################
plume_name = 'Plume 1'
####################################
all_data_dir = source + case_name + '_' + str(np.around(cen_lon, decimals=4)) + '_' + str(np.around(cen_lat, decimals=4)) + '_interpolated_' + str(pixel_size) + '_' + str(delta_lonlat) + '/'
specific_main_dates = [#dt(2020, 1, 3).date(),
                       dt(2020, 1, 4).date(),
                       dt(2020, 1, 5).date(),
                       dt(2020, 1, 6).date(),
                       dt(2020, 1, 7).date(),
                       #dt(2020, 1, 8).date(),
                       #dt(2020, 1, 9).date()
                        ]
Plume1_input = {'plume_name': plume_name,
                'cen_lon': cen_lon,
                'cen_lat': cen_lat,
                'delta_lonlat': delta_lonlat,
                'pixel_size': pixel_size,
                'all_data_dir': all_data_dir,
                'specific_main_dates': specific_main_dates}

## 2. Interpolate

Having downloaded the data, an interpolation, or re-gridding, is required to match the pixels of main and reference observations. This is done for only a portion of the complete scene that was downloaded. This time-consuming step is only required once, but it can be done multiple times for one case with different centers or scene widths (delta_lonlat) if the exact source location is unknown or further zooming is required.

In [ ]:
# Interpolate all files to a regular grid

interpolated_dir = all_data_dir

if not os.path.isdir(interpolated_dir):
    os.makedirs(interpolated_dir)

files_to_interpolate = os.listdir(original_file_dir)
files_to_interpolate.sort()
print(delta_lonlat)

####################################
##           INTERPOLATE          ##
####################################

# Define area of interest
grid_of_interest = [cen_lon-delta_lonlat, cen_lon+delta_lonlat,
                    cen_lat-delta_lonlat, cen_lat+delta_lonlat]

# Desired grid for interpolation later on
lon_reg = np.arange(grid_of_interest[0], grid_of_interest[1]+pixel_size, pixel_size)
lat_reg = np.arange(grid_of_interest[2], grid_of_interest[3]+pixel_size, pixel_size)
lon_reg, lat_reg = np.meshgrid(lon_reg, lat_reg)

print('Interpolating ', len(files_to_interpolate), ' files ... ')

for file in files_to_interpolate:
    lon, lat, s6, s5, s4, s3, s2, s1, landscape, sza, vza, lon_tx, lat_tx = read_full_slstr_data(original_file_dir + file)
    lon, lat, [s6, s5, s4, s3, s2, s1, landscape] = get_data_on_grid_of_interest(lon, lat, [s6, s5, s4, s3, s2, s1, landscape], grid_of_interest)
    water_mask = get_inland_water_mask(landscape)
    lon_tx, lat_tx, [sza, vza] = get_data_on_grid_of_interest(lon_tx, lat_tx, [sza, vza], grid_of_interest)
    sza_average = np.mean(sza)
    vza_average = np.mean(vza)


    try:
        [s6, s5, s4, s3, s2, s1, water_mask] = perform_interpolation(lon, lat, lon_reg, lat_reg, [s6, s5, s4, s3, s2, s1, water_mask])
    except:
        print(f'Error in file {file}. Skipped')
        continue

    [s6, s5, s4, s3, s2, s1, water_mask] = set_nan_inf_neg_to_zero([s6, s5, s4, s3, s2, s1, water_mask])

    # Save lon, lat, s6, s5, s4, s3, s2, s1 and water_mask to interpolated dir.
    save_to_path = interpolated_dir + file[:-5]
    if not os.path.isdir(save_to_path):
                os.makedirs(save_to_path)
    np.save(save_to_path + '/lon_interpolated.npy', lon_reg)
    np.save(save_to_path + '/lat_interpolated.npy', lat_reg)
    np.save(save_to_path + '/S6_interpolated.npy', s6)
    np.save(save_to_path + '/S5_interpolated.npy', s5)
    np.save(save_to_path + '/S4_interpolated.npy', s4)
    np.save(save_to_path + '/S3_interpolated.npy', s3)
    np.save(save_to_path + '/S2_interpolated.npy', s2)
    np.save(save_to_path + '/S1_interpolated.npy', s1)
    np.save(save_to_path + '/water_interpolated.npy', water_mask)
    np.save(save_to_path + '/SZA_average.npy', sza_average)
    np.save(save_to_path + '/VZA_average.npy', vza_average)

print(' ... Finished interpolation')

## 3. Process

### Settings:

*   **auxiliary_bands_to_consider**: bands to consider in the auxiliary mask

*   **correlation_minimum**: minimum auxiliary band correlation to consider it in the mask

*   **delR_mask_limit**: maximum DelR_MBSP value to consider in that filter, as to remove already signals that cannot be due to methane

*   **delta_lonlat_tropomi**: half width of tropomi scene

*   **destripe**: whether to apply the destriping procedure

*   **filter_auxiliary_mask**: whether to apply the structural similarity mask between auxiliary bands to identify noise regions

*   **filter_delR_mask**: whether to apply the structural similarity mask between DelR_MBSP to identify regions of interest (potential methane plumes)

*   **filter_median**: whether to include a median filter

*   **filter_top_xperc**: whether to include a top x percentile filter

*   **img_quality_S3**: quality of background in S3 standard image

*   **input_dict**: dictionary containing the cases with their corresponding case specific inputs as defined in Inputs

*   **median_filter_size**: size of median filter

*   **min_delR**: minimum value of DelR_MBMP to consider the results valid

*   **min_vis_corr**: minimum correlation with main observation in the visible band (called S1) to consider a reference observation

*   **number_of_references_original**: the number of best reference observations that should be used to construct the final reference observations by taking the pixel-wise median. If not enough valid once are present, less will be used.

*   **reference_day_range**: range of days ahead and before the main date that are considered valid reference observations. Optional: (defaults are present in Sentinel3Class.py)

*   **save_results**: whether to save the standard images and Pickle files

*   **SHOW_STANDARD_S3**: whether to show the Sentinel 3 standard image

*   **top_xperc**: the value for x in the filter

*   **VERBOSE**: whether to print information during the processing

In [ ]:
start_time = time.time()

####################################
##   DEFINE PROCESSING SETTINGS   ##
####################################

##### Reference date selection criteria -> optional parameters when initializing sentinel3() object
min_vis_corr = 0.3
min_delR = -5.
number_of_references_original = 5
reference_day_range = 30

##### Filter settings

destripe = True

filter_delR_mask = True
delR_mask_limit = 0.1

filter_auxiliary_mask = True
correlation_minimum = 0.5
auxiliary_bands_to_consider = [1, 2, 3] # 0=S4, 1=S3, 2=S2, 3=S1

filter_top_xperc = False
top_xperc = 0

filter_median = False
median_filter_size = 5

####################################
##      SET DESIRED RESULTS       ##
####################################

SHOW_STANDARD_S3 = True

save_results = True

img_quality_S3 = 14

VERBOSE = True

####################################
##         LOAD CASE INPUTS       ##
####################################

input_dict = {
    'Plume1': Plume1_input,
     }

########################################
# Create output folders

if save_results:
  if not os.path.isdir('Pickle_files/Sentinel_3'):
      os.makedirs('Pickle_files/Sentinel_3')
      os.makedirs('Pickle_files/Sentinel_2')
  if not os.path.isdir('StandardImages/Sentinel_3'):
      os.makedirs('StandardImages/Sentinel_3')
      os.makedirs('StandardImages/Sentinel_2')

all_omegas = []
all_filtered_omegas = []

# Loop over the locations
for key in input_dict.keys():

    if VERBOSE:
        print();print('###################################################');print()
        print(key)
        print();print('###################################################');print()

    plume_name = input_dict[key]['plume_name']
    cen_lon = input_dict[key]['cen_lon']
    cen_lat = input_dict[key]['cen_lat']
    delta_lonlat = input_dict[key]['delta_lonlat']
    pixel_size = input_dict[key]['pixel_size']
    all_data_dir = input_dict[key]['all_data_dir']
    specific_main_dates = input_dict[key]['specific_main_dates']

    if VERBOSE:
        print(all_data_dir)

    if specific_main_dates == 'All':
        if VERBOSE:
            print('Processing all dates')
        main_file_list = os.listdir(all_data_dir)
        for i in range(len(main_file_list)):
          main_file_list[i] = all_data_dir + main_file_list[i]

    else:
        if VERBOSE:
            print('Processing specific dates only')
        main_file_list = get_specific_date_files(specific_main_dates, all_data_dir)

    # Loop over main date files of current location
    for main_file in main_file_list:

        S3_object = sentinel3(plume_name,
                              all_data_dir,
                              main_file,
                              cen_lon,
                              cen_lat,
                              delta_lonlat=delta_lonlat,
                              pixel_size=pixel_size,
                              verbose=VERBOSE,
                              number_of_references_original=number_of_references_original,
                              reference_day_range=reference_day_range
                             )

        S3_object.get_correlation_based_reference_data()
        S3_object.perform_multi_band_multi_pass_method()
        S3_object.apply_filters_and_calculate_omega(destripe=destripe,
                                                    filter_delR_mask=filter_delR_mask,
                                                    delR_mask_limit=delR_mask_limit,
                                                    filter_auxiliary_mask=filter_auxiliary_mask,
                                                    correlation_minimum=correlation_minimum,
                                                    auxiliary_bands_to_consider=auxiliary_bands_to_consider,
                                                    filter_top_xperc=filter_top_xperc,
                                                    top_xperc=top_xperc,
                                                    filter_median=filter_median,
                                                    median_filter_size=median_filter_size
                                                   )

        S3_object.create_standard_images_and_save_data_to_pickle_files(S3_img_quality=img_quality_S3,
                                                                           S3_show=SHOW_STANDARD_S3,
                                                                           retrieve_tropomi=False,
                                                                           retrieve_S2=False,
                                                                           save_results=save_results)

        all_omegas.append(S3_object.omega)
        all_filtered_omegas.append(S3_object.omega_filtered)

########################################################################
# END
########################################################################

end_time = time.time()
print()
print('###')
print()
print('Script duration: ', (end_time-start_time), ' seconds')

In [ ]:
sum_omega = np.sum(all_omegas, axis=0)
sum_filtered_omega = np.sum(all_filtered_omegas, axis=0)

S3_object.create_single_plot(sum_omega, sum_filtered_omega, 'Total methane emitted')